In [45]:
import pandas as pd
import numpy as np
import os, sys
import glob, re, time

from collections import Counter

In [4]:
os.getcwd()

'/Volumes/LaCie/Documents/repos/particle_filter'

In [2]:
class particle_filter:
    """
    n = number of particles to maintain at each step in time
    """
    def __init__(n=50):
        self.n = n
        
    

'0.24.2'

In [15]:
## it seems i only have beijing data (10K) or sf taxi data (500)
# Read in taxi data
# Each data of different length; ideal use case for pyspark
# Note, the OSM extract basemap data has POI info as well (https://download.bbbike.org)
# also try uber h3 spatial index
trace_dir = '/Volumes/LaCie/datasets/ms_taxi/taxi_log_2008_by_id/'   # MS Taxi
basemap_dir = '/Volumes/LaCie/datasets/Beijing-shp/shape/'

trace_dir = '/Volumes/LaCie/datasets/cabspottingdata/'   # CRAWDAD cabspotting
basemap_dir = '/Volumes/LaCie/datasets/SanFrancisco-shp/shape/'

In [28]:
%%time
all_files = [f for f in os.listdir(trace_dir) if re.match(r'new_.*\.txt', f)]  # glob.glob(trace_dir + "new_*.txt")

CPU times: user 2.07 ms, sys: 906 µs, total: 2.97 ms
Wall time: 2.31 ms


In [51]:
%%time
# 20.9s cabspottingdata; can try spark or dask? or multiprocessing, joblib
trace_list = []
for file_ in all_files:
    file_df = pd.read_csv(file_, sep=" ", index_col=None, header=None, 
                          names=['lat', 'long', 'occupancy', 'time'])
    trace_list.append(file_df)

# concatenate all dfs into one
trace_df = pd.concat(trace_list, ignore_index=True)

CPU times: user 8.28 s, sys: 1.87 s, total: 10.2 s
Wall time: 14.3 s


In [ ]:
trace_df.loc[:, ["time"]] = pd.to_datetime(trace_df.time, origin="unix", unit='s')

In [59]:
"""
cabspotting:

latitude and longitude are in decimal degrees, 
occupancy shows if a cab has a fare (1 = occupied, 0 = free) and 
time is in UNIX epoch format
"""
trace_df.dtypes

lat                 float64
long                float64
occupancy             int64
time         datetime64[ns]
dtype: object

In [60]:
trace_df.head()

lat       long  occupancy                time
0  37.75134 -122.39488          0 2008-06-10 07:58:07
1  37.75136 -122.39527          0 2008-06-10 07:57:39
2  37.75199 -122.39460          0 2008-06-10 07:55:40
3  37.75080 -122.39346          0 2008-06-10 07:54:49
4  37.75015 -122.39256          0 2008-06-10 07:50:37